# iPRules

# Statics

In [14]:
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.utils import Bunch
from sklearn.datasets._base import load_csv_data
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import csv

## Load Dataset

In [15]:
# Load Dataset
#iris
#dataset = load_iris()
target_value_name = 'class'

# Mushrooms
filename = 'mushrooms'
target_true = 'p'
target_false = 'e'
test_size = 0.33
min_number_class_per_node = 3


data_file_name = f'../../data/{filename}.csv'
pandas_dataset = pd.read_csv(data_file_name)

pandas_dataset.columns = [sub.replace('%', '') for sub in pandas_dataset.columns]
target_value_name = pandas_dataset.columns[-1]

pandas_dataset[target_value_name] = pandas_dataset[target_value_name].map({target_false:0,target_true:1})

pandas_dataset.columns = [sub.replace('-', '_').replace(' ', '').replace('class', 'target_value') for sub in pandas_dataset.columns]
target_value_name = pandas_dataset.columns[-1]
feature_names = pandas_dataset.columns[0:-1]


#dataset.feature_names = [sub.replace(' ', '').replace('(cm)', '') for sub in dataset.feature_names]

pandas_dataset.head()

,cap_shape,cap_surface,cap_color,bruises3F,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,...,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat,target_value
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,1
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,0
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,0
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,1
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,0


# One Hot Encoding

In [16]:
enc = OneHotEncoder(sparse_output=False)
encoded_array = enc.fit_transform(pandas_dataset.loc[:,feature_names])
encoded_feature_names = enc.get_feature_names_out()
df_encoded = pd.DataFrame(encoded_array,columns=encoded_feature_names)
encoded_pandas_dataset = pd.concat([df_encoded, pandas_dataset],axis=1)
encoded_pandas_dataset.drop(labels= feature_names,axis=1,inplace=True)
encoded_pandas_dataset.head()

,cap_shape_b,cap_shape_c,cap_shape_f,cap_shape_k,cap_shape_s,cap_shape_x,cap_surface_f,cap_surface_g,cap_surface_s,cap_surface_y,...,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w,target_value
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [17]:
#print(pandas_dataset.shape)

# One hot encoding + not doing anything in the rest
#ct = make_column_transformer(
#    (OneHotEncoder(), feature_names),
#    n_jobs=3,
#    remainder='passthrough',
#    sparse_threshold=0)

#pandas_dataset_encoded = ct.fit_transform(pandas_dataset).T

#print(pandas_dataset_encoded[0])

#column_names = (ct.named_transformers_["onehotencoder"].get_feature_names_out().tolist()
#               + [target_value_name])

#encoded_dataset = pd.DataFrame(pandas_dataset_encoded, column_names)

#encoded_dataset.head()

In [18]:
X = encoded_pandas_dataset[encoded_feature_names]
y = encoded_pandas_dataset[target_value_name]

encoded_dataset = Bunch(
        data=X.to_numpy(),
        target=y.to_numpy(),
        target_names=target_value_name,
        feature_names=X.columns
)

#X_display = pandas_dataset.drop([target_value_name], axis=1)
#y_display = pandas_dataset[target_value_name]

#encoded_pandas_dataset = pd.DataFrame(data= np.c_[dataset['data'], dataset['target']], columns= list(dataset['feature_names']) + ['target'])
#encoded_pandas_dataset.head()


# Divide dataset

In [19]:
#Define dataset
X_train, X_test, y_train, y_test = train_test_split(encoded_dataset.data, encoded_dataset.target, test_size=test_size, random_state=1)
encoded_train_pandas_dataset = pd.DataFrame(data= np.c_[X_train, y_train], columns= list(encoded_dataset['feature_names']) + [target_value_name])
encoded_test_pandas_dataset = pd.DataFrame(data= np.c_[X_test, y_test], columns= list(encoded_dataset['feature_names']) + [target_value_name])
print()
print('Sizes (without target):')
print(f'Original size {encoded_dataset.data.shape}')
print(f'Train size {X_train.shape}')
print(f'Test size {X_test.shape}')


Sizes (without target):
Original size (8124, 117)
Train size (5443, 117)
Test size (2681, 117)


## Define Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, make_scorer, recall_score, accuracy_score

# Define scorer
ensemble = RandomForestClassifier(n_estimators=100)

## iPRules

In [21]:
from iPRules.iPRules import iPRules

# initialize
tree = iPRules(
                base_ensemble=ensemble,
                feature_names=encoded_dataset.feature_names,
                target_value_name = encoded_dataset.target_names,
                chi_square_probability = 0.95,
                scale_feature_coefficient = 0.85,
                min_number_class_per_node = min_number_class_per_node
            )

# Fit model
tree.fit(encoded_train_pandas_dataset, X_train, y_train)

Fit Ensemble Model
Extract feature importance list
Generate new tree based on list
Generate obtained patterns tree


iPRules(base_ensemble=RandomForestClassifier(),
        feature_names=Index(['cap_shape_b', 'cap_shape_c', 'cap_shape_f', 'cap_shape_k',
       'cap_shape_s', 'cap_shape_x', 'cap_surface_f', 'cap_surface_g',
       'cap_surface_s', 'cap_surface_y',
       ...
       'population_s', 'population_v', 'population_y', 'habitat_d',
       'habitat_g', 'habitat_l', 'habitat_m', 'habitat_p', 'habitat_u',
       'habitat_w'],
      dtype='object', length=117),
        target_value_name='target_value')

# Print Model

In [22]:
print(tree)

> ------------------------------
> iPRules:
> Number of patterns17:
> ------------------------------
Regla 0: odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0.0 & ring_type_p == 1.0 & gill_color_b == 0.0 & stalk_surface_below_ring_k == 0.0 & gill_spacing_c == 0.0 & population_v == 1.0 & bruises3F_f
Regla 1: odor_n == 1.0 & odor_f == 0.0 & gill_size_n == 0.0 & gill_size_b == 1.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0.0 & ring_type_p
Regla 2: odor_n == 1.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h
Regla 3: odor_n == 1.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k
Regla 4: odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k
Regla 5: odor_n == 0.0 & odor_f
Regla 6: odor_n == 0.0 & odor_f == 0.0 & 

# Predict

In [23]:
#Divide las combinaciones en si el patrón lo muestran en supervivencia o fallecimiento.
negative_categorized_rules, positive_categorized_rules = tree.categorize_patterns(encoded_train_pandas_dataset, 0.9)

print(len(negative_categorized_rules))
print(len(positive_categorized_rules))

print(f'Negative Rules {len(negative_categorized_rules)} ')
sum_total = 0
sum_specific = 0
all_rules = []
for rule, number_specific_case, number_total in negative_categorized_rules:
    sum_total += number_total
    sum_specific += number_specific_case
    all_rules += [rule]
    print(f'{rule} \n')

print(f'Positive Rules {len(positive_categorized_rules)} ')
for rule, number_specific_case, number_total in positive_categorized_rules:
    sum_total += number_total
    sum_specific += number_specific_case
    all_rules += [rule]
    print(f'{rule} \n')

12
15
Negative Rules 12 
odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0.0 & ring_type_p == 1.0 & gill_color_b == 0.0 & stalk_surface_below_ring_k == 0.0 & gill_spacing_c == 0.0 & population_v == 1.0 & bruises3F_f == 1 

odor_n == 1.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 1 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 1 

odor_n == 0.0 & odor_f == 1 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0.0 & ring_type_p == 1.0 & gill_color_b == 0.0 & stalk_surface_below_ring_k == 0.0 & gill_spacing_c == 0.0 & populatio

In [27]:
sortedList = sorted(all_rules)
print(f' Rules {len(sortedList)} ')
for rules in sortedList:
    print(f'{rules} \n')

 Rules 27 
odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 0 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 0.0 & gill_size_b == 1.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 0.0 & gill_size_b == 1.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 1 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0.0 & ring_type_p == 0 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gill_size_b == 0.0 & spore_print_color_h == 0.0 & stalk_surface_above_ring_k == 0.0 & ring_type_p == 1.0 & gill_color_b == 0.0 & stalk_surface_below_ring_k == 0.0 & gill_spacing_c == 0.0 & population_v == 0 

odor_n == 0.0 & odor_f == 0.0 & gill_size_n == 1.0 & gil

In [25]:

y_pred_test = ensemble.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_test)
#Returns death and survival patterns where there's a % of the total cases of that specific type.


negative_acc = 0
negative_total = 0

positive_acc = 0
positive_total = 0

data_file_name = f'../../data/{filename}_combinations_negatives.csv'

for pattern in negative_categorized_rules:
    negative_acc += pattern[1]
    negative_total += pattern[2]

for pattern in positive_categorized_rules:
    positive_acc += pattern[1]
    positive_total += pattern[2]


with open(f'../../data/{filename}_combinations_negatives.csv','w',encoding='UTF8') as file:
    writer = csv.writer(file)
    writer.writerow(['Combinacion','Casos_encontrados','Casos_totales'])
    for pattern in negative_categorized_rules:
        writer.writerow(pattern)
    file.close()

with open(f'../../data/{filename}_combinations_positives.csv','w',encoding='UTF8') as file:
    writer = csv.writer(file)
    writer.writerow(['Combinacion','Casos_encontrados','Casos_totales'])
    for pattern in positive_categorized_rules:
        writer.writerow(pattern)
    file.close()


print('Casualties accuracy:',str("{:.2f}".format(100*(negative_acc/negative_total)))+'%')
print('Survival accuracy:',str("{:.2f}".format(100*(positive_acc/positive_total)))+'%')
print('Top model RF accuracy:',str("{:.2f}".format(100*accuracy))+'%')

Casualties accuracy: 96.68%
Survival accuracy: 97.06%
Top model RF accuracy: 100.00%


In [26]:
tree_preds = tree.predict(X_test)
print('The accuracy of the Tree model is :\t',metrics.accuracy_score(tree_preds,y_test))

AttributeError: 'NoneType' object has no attribute 'predict'

# SHAP explainer

In [ ]:
import shap

# Create Tree Explainer object that can calculate shap values
explainer = shap.TreeExplainer(ensemble)

# Evaluate SHAP values
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_train, feature_names=encoded_dataset.feature_names)

# Relation

In [ ]:
shap.summary_plot(shap_values, X_train, feature_names=encoded_dataset.feature_names, plot_type="dot")